# Lab Exercise 3

Exercise description in German.

## Kalman-Filter mit asynchronen Messdaten

### Problemstellung

In einem Fahrzeug wird mittels eines GPS-Sensors die Position und über einen Inertialsensor die Beschleunigung erfasst. Die Datenerfassung findet asynchron und in nicht äquidistanten Zeitabständen statt.

In [ ]:
import pandas as pd

async_data = pd.read_csv(
    "data/Asynchron_data_in.dat",
    sep=" ",
)
async_data

In [ ]:
plot_s, plot_a = async_data.plot(
    y=["position", "acceleration"],
    kind="line",
    grid=True,
    xlabel="Zeit [s]",
    title="Eingangsdaten",
    subplots=True,
    style="x",
)
plot_s.set_ylabel("Position [m]")
plot_a.set_ylabel("Beschleunigung [m/s²]")

### Aufgabenstellung

Entwerfen Sie ein geeignetes Kalman-Filter, welches in der Lage ist die nicht zeitäquidistanten Größen zu fusionieren.

Hinweis zur Bearbeitung: Überlegen sich welches klassisches Kalman-Filter für die Aufgabe geeignet ist und was man an dem in der Vorlesung behandelten Kalman-Filter ändern müsste, sodass man den nicht äquidistant Abtastintervallen gerecht wird.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

y = async_data[["position", "acceleration"]].to_numpy()
u = np.zeros((y.size, 1))
t = async_data["time"].to_numpy()

#### Gamma-Filter

$$\boldsymbol{x}(k + 1) = \begin{bmatrix}
1 & T_s & \frac{T_s^2}{2} \\
0 & 1 & T_s \\
0 & 0 & 1 \\
\end{bmatrix} + \begin{bmatrix}
0 \\
0 \\
0 \\
\end{bmatrix} \cdot \boldsymbol{u}(k) + \begin{bmatrix}
\frac{T_s^2}{2} \\
T_s \\
1 \\
\end{bmatrix} \cdot \boldsymbol{z}(k)$$

$$y(k) = \begin{bmatrix}
1 & 0 & 0 \\
\end{bmatrix} \cdot \begin{bmatrix}
s(k) \\
v(k) \\
a(k) \\
\end{bmatrix} + \begin{bmatrix}
0 \\
0 \\
0 \\
\end{bmatrix} \cdot \boldsymbol{u}(k)$$

In [ ]:
stable_s = y[t < 2, 0]
stable_a = y[(t > 2) & (t < 8), 1]
R_s = stable_s[~np.isnan(stable_s)].var()
R_a = stable_a[~np.isnan(stable_a)].var()
Q = 0.0005
print(f"{R_s=:g}, {R_a:g}")
print(f"{Q=:g}")

Bd = np.array([[0], [0], [0]])
D = 0

In [ ]:
y_s = y[:, 0]
y_a = y[:, 1]
x_tilde = np.array([[y_s[~np.isnan(y_s)][0]], [0], [0]])
P_tilde = 5 * np.eye(3)

d_y = np.zeros_like(y_s)

s = np.zeros_like(y_s)
v = np.zeros_like(y_s)
a = np.zeros_like(y_s)
K1 = np.zeros_like(y_s)
K2 = np.zeros_like(y_s)
K3 = np.zeros_like(y_s)
P_tilde1 = np.zeros_like(y_s)
P_tilde2 = np.zeros_like(y_s)
P_tilde3 = np.zeros_like(y_s)
P_tilde4 = np.zeros_like(y_s)
P_tilde5 = np.zeros_like(y_s)
P_tilde6 = np.zeros_like(y_s)
P_tilde7 = np.zeros_like(y_s)
P_tilde8 = np.zeros_like(y_s)
P_tilde9 = np.zeros_like(y_s)
for i, (s_i, a_i) in enumerate(y[1:, :]):
    Ts = t[i + 1] - t[i]
    Ad = np.array([[1, Ts, 0.5 * (Ts**2)], [0, 1, Ts], [0, 0, 1]])
    Gd = np.array([[0.5 * (Ts**2)], [Ts], [1]])

    if np.isnan(s_i):
        C = np.array([[0, 0, 1]])
        R = R_a
        ym = a_i
    else:
        C = np.array([[1, 0, 0]])
        R = R_s
        ym = s_i

    x_hat = Ad @ x_tilde + (Bd @ u[i]).reshape(-1, 1)
    P_hat = Ad @ P_tilde @ Ad.T + (Gd * Q) @ Gd.T

    d_y[i] = ym - (C @ x_hat + D * u[i])
    k = C @ P_hat @ C.T + R
    if k.ndim >= 2:
        K = P_hat @ C.T @ np.linalg.pinv(k)
    else:
        K = P_hat @ C.T / k
    x_tilde = x_hat + (K * d_y[i]).reshape(-1, 1)
    P_tilde = (np.eye(Bd.size) - K @ C) @ P_hat

    s[i], v[i], a[i] = x_tilde
    K1[i], K2[i], K3[i] = K
    P_tilde1[i], P_tilde2[i], P_tilde3[i] = P_tilde[0, :]
    P_tilde4[i], P_tilde5[i], P_tilde6[i] = P_tilde[1, :]
    P_tilde7[i], P_tilde8[i], P_tilde9[i] = P_tilde[2, :]
fig, (ax_s, ax_v, ax_a) = plt.subplots(3, 1, figsize=(15, 7))

ax_s.plot(t, y_s, "ko")
ax_s.plot(t, s, "b--")
ax_s.plot(t, s + 3 * np.sqrt(P_tilde1), "b-")
ax_s.plot(t, s - 3 * np.sqrt(P_tilde1), "b-")
ax_s.grid(True)
ax_s.set_xlabel("Zeit [s]")
ax_s.set_ylabel("Position [m]")

ax_v.plot(t, v, "r-*")
ax_v.plot(t, v + 3 * np.sqrt(P_tilde5), "r-")
ax_v.plot(t, v - 3 * np.sqrt(P_tilde5), "r-")
ax_v.grid(True)
ax_v.set_xlabel("Zeit [s]")
ax_v.set_ylabel("Geschwindigkeit [m/s]")

ax_a.plot(t, y_a, "ko")
ax_a.plot(t, a, "g-*")
ax_a.plot(t, a + 3 * np.sqrt(P_tilde9), "g-")
ax_a.plot(t, a - 3 * np.sqrt(P_tilde9), "g-")
ax_a.grid(True)
ax_a.set_xlabel("Zeit [s]")
ax_a.set_ylabel("Beschleunigung [m/s²]")

fig.suptitle("Gamma-Filter")
fig.tight_layout()